In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageDraw, ImageOps, ImageFilter, ImageEnhance
import cv2
import numpy as np
import matplotlib.pyplot as plt
from twilio.rest import Client
import pyttsx3
import geocoder
from googlesearch import search
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

# Twilio credentials
account_sid = 'your_twilio_account_sid'
auth_token = 'your_twilio_auth_token'
twilio_client = Client(account_sid, auth_token)

# Initialize the text-to-speech engine
tts_engine = pyttsx3.init()

def capture_image(filename='captured_image.jpg'):
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        messagebox.showerror("Error", "Could not access the webcam.")
        return

    ret, frame = cap.read()
    if ret:
        cv2.imwrite(filename, frame)
        messagebox.showinfo("Info", f"Image saved as {filename}")
    else:
        messagebox.showerror("Error", "Could not capture an image.")

    cap.release()
    cv2.destroyAllWindows()

def apply_realistic_sunglasses_filter():
    image_path = 'captured_image.jpg'
    image = Image.open(image_path).convert('RGBA')
    width, height = image.size

    overlay = Image.new('RGBA', image.size, (0, 0, 0, 0))
    draw = ImageDraw.Draw(overlay)

    glasses_width = width // 3
    glasses_height = height // 10
    bridge_width = width // 10
    bridge_height = glasses_height // 2

    left_eye_center = (width // 3, height // 3)
    right_eye_center = (2 * width // 3, height // 3)
    bridge_center = (width // 2, height // 3 + glasses_height // 2)

    left_bbox = [left_eye_center[0] - glasses_width // 2, left_eye_center[1] - glasses_height // 2,
                 left_eye_center[0] + glasses_width // 2, left_eye_center[1] + glasses_height // 2]
    draw.ellipse(left_bbox, fill=(0, 0, 0, 128))

    right_bbox = [right_eye_center[0] - glasses_width // 2, right_eye_center[1] - glasses_height // 2,
                  right_eye_center[0] + glasses_width // 2, right_eye_center[1] + glasses_height // 2]
    draw.ellipse(right_bbox, fill=(0, 0, 0, 128))

    bridge_bbox = [bridge_center[0] - bridge_width // 2, bridge_center[1] - bridge_height // 2,
                   bridge_center[0] + bridge_width // 2, bridge_center[1] + bridge_height // 2]
    draw.rectangle(bridge_bbox, fill=(0, 0, 0, 128))

    image_with_sunglasses = Image.alpha_composite(image, overlay)
    image_with_sunglasses = image_with_sunglasses.convert('RGB')
    image_with_sunglasses.save('realistic_sunglasses_filter.jpg')
    image_with_sunglasses.show()

def display_filters():
    image_path = r"C:\Users\HP\Downloads\bird image.jpg"
    image = Image.open(image_path)

    blurred_image = image.filter(ImageFilter.BLUR)
    sharpened_image = image.filter(ImageFilter.SHARPEN)
    edge_enhanced_image = image.filter(ImageFilter.EDGE_ENHANCE)
    grayscale_image = image.convert("L")
    enhancer = ImageEnhance.Brightness(image)
    brightened_image = enhancer.enhance(1.5)

    plt.figure(figsize=(12, 8))

    def display_image(image, title):
        plt.imshow(image)
        plt.title(title)
        plt.axis('off')

    plt.subplot(2, 3, 1)
    display_image(image, "Original")

    plt.subplot(2, 3, 2)
    display_image(blurred_image, "Blurred")

    plt.subplot(2, 3, 3)
    display_image(sharpened_image, "Sharpened")

    plt.subplot(2, 3, 4)
    display_image(edge_enhanced_image, "Edge Enhanced")

    plt.subplot(2, 3, 5)
    display_image(grayscale_image, "Grayscale")

    plt.subplot(2, 3, 6)
    display_image(brightened_image, "Brightened")

    plt.show()

def send_sms():
    from_phone_number = '+16302803609'
    to_phone_number = '+917000622556'
    message_body = 'Hello from Python!'
    message = twilio_client.messages.create(
        body=message_body,
        from_=from_phone_number,
        to=to_phone_number
    )
    messagebox.showinfo("Info", f"Message sent with SID: {message.sid}")

def make_call():
    from_phone_number = '+12566158865'
    to_phone_number = '+917000622556'
    twiml_url = 'http://demo.twilio.com/docs/voice.xml'
    call = twilio_client.calls.create(
        to=to_phone_number,
        from_=from_phone_number,
        url=twiml_url
    )
    messagebox.showinfo("Info", f"Call initiated with SID: {call.sid}")

def send_whatsapp_message():
    account_sid = 'ACe94c52f004d4c70e599a4603821feaf1'
    auth_token = '4290c6a289ef7ce82321e142ee5b0b70'
    client = Client(account_sid, auth_token)
    from_whatsapp_number = 'whatsapp:+14155238886'
    to_whatsapp_number = 'whatsapp:+917000622556'
    message = client.messages.create(
        body='Hello from Python via WhatsApp!',
        from_=from_whatsapp_number,
        to=to_whatsapp_number
    )
    messagebox.showinfo("WhatsApp Message Status", f"Message sent with SID: {message.sid}")


def get_location():
    g = geocoder.ip('me')
    location_info = f"Latitude: {g.lat}\nLongitude: {g.lng}\nAddress: {g.address}"
    messagebox.showinfo("Location Info", location_info)

def text_to_speech():
    text = "Heyy, this is Ishika"
    tts_engine.say(text)
    tts_engine.runAndWait()

def search_google():
    query = "top singers"
    results = search(query, num_results=5)
    results_info = "\n".join(f"{i + 1}. {result}" for i, result in enumerate(results))
    messagebox.showinfo("Search Results", results_info)

def adjust_volume():
    devices = AudioUtilities.GetSpeakers()
    interface = devices.Activate(IAudioEndpointVolume.iid, CLSCTX_ALL, None)
    volume = cast(interface, POINTER(IAudioEndpointVolume))
    volume.SetMasterVolumeLevel(-20.0, None)
    messagebox.showinfo("Info", "Volume adjusted")

def create_image():
    color_image = np.zeros((100, 100, 3), dtype=np.uint8)
    color_image[40:60, 40:60, 0] = 255
    plt.imshow(color_image)
    plt.title("Color Image")
    plt.axis('off')
    plt.show()

def detect_face(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    if len(faces) == 0:
        return None
    x, y, w, h = faces[0]
    return (x, y, w, h)

def overlay_face_on_image(original_image_path, face_image_path, face_bbox):
    original_image = Image.open(original_image_path)
    face_image = Image.open(face_image_path)
    face_image = face_image.resize((face_bbox[2], face_bbox[3]))
    overlay = Image.new('RGBA', original_image.size, (0, 0, 0, 0))
    overlay.paste(face_image, (face_bbox[0], face_bbox[1]))
    combined_image = Image.alpha_composite(original_image.convert('RGBA'), overlay)
    combined_image = combined_image.convert('RGB')
    combined_image.save('overlayed_face_image.jpg')
    combined_image.show()

def process_image():
    image_path = 'captured_image.jpg'
    face_bbox = detect_face(image_path)
    if face_bbox:
        image = Image.open(image_path)
        cropped_face = image.crop((face_bbox[0], face_bbox[1], face_bbox[0] + face_bbox[2], face_bbox[1] + face_bbox[3]))
        cropped_face.save('cropped_face.jpg')
        overlay_face_on_image(image_path, 'cropped_face.jpg', face_bbox)
    else:
        messagebox.showerror("Error", "No face detected.")

# Create the main Tkinter window
root = tk.Tk()
root.title("Multifunctional App")
root.configure(bg='lightblue')

# Add welcome message and name at the top
tk.Label(root, text="Ishika's Application", font=('Helvetica', 16, 'bold'), bg='lightblue').pack(pady=10)
tk.Label(root, text="Welcome to my project!", font=('Helvetica', 14), bg='lightblue').pack(pady=5)


tk.Button(root, text="Capture Image", command=capture_image, bg='lightgreen').pack(pady=10, padx=25)
tk.Button(root, text="Apply Sunglasses Filter", command=apply_realistic_sunglasses_filter, bg='lightyellow').pack(pady=10, padx=25)
tk.Button(root, text="Display Filters", command=display_filters, bg='lightcoral').pack(pady=10, padx=25)
tk.Button(root, text="Send SMS", command=send_sms, bg='lightpink').pack(pady=10, padx=25)
tk.Button(root, text="Make Call", command=make_call, bg='lightgoldenrodyellow').pack(pady=10, padx=25)
tk.Button(root, text="Send WhatsApp Message", command=send_whatsapp_message, bg='lightblue').pack(pady=10, padx=25)
tk.Button(root, text="Get Location", command=get_location, bg='lightcyan').pack(pady=10, padx=25)
tk.Button(root, text="Text to Speech", command=text_to_speech, bg='lightsteelblue').pack(pady=10, padx=25)
tk.Button(root, text="Google Search", command=search_google, bg='lightgray').pack(pady=10, padx=25)
tk.Button(root, text="Adjust Volume", command=adjust_volume, bg='lightsalmon').pack(pady=10, padx=25)
tk.Button(root, text="Create Color Image", command=create_image, bg='lightseagreen').pack(pady=10, padx=25)
tk.Button(root, text="Process Image", command=process_image, bg='lightpink').pack(pady=10, padx=25)
tk.Button(root, text="EXIT", command=exit, bg='lightgray').pack(pady=10, padx=25)

root.mainloop()